In [15]:
#いろいろインポート
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ
place='kiryu'

In [16]:
#モデルスコアデータの読み込み
#score_df = pd.read_csv('../csv/model_score_date_{}.csv'.format(place))#dateアリのモデル
score_df = pd.read_csv('../csv/{}_model_score_normal.csv'.format(place))#dateなしのモデル
score_df=score_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
display(score_df)

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,374060.0,423200.0,308.0,4232.0,88.388469,7.277883,308.0
1,1.0,4.0,52.0,374060.0,423200.0,308.0,4232.0,88.388469,7.277883,308.0
2,1.0,4.0,53.0,374060.0,423100.0,308.0,4231.0,88.409359,7.279603,308.0
3,1.0,4.0,54.0,374060.0,423100.0,308.0,4231.0,88.409359,7.279603,308.0
4,1.0,4.0,55.0,374060.0,422900.0,308.0,4229.0,88.451170,7.283046,308.0
...,...,...,...,...,...,...,...,...,...,...
13855,28.0,8.0,145.0,6650.0,19500.0,20.0,195.0,34.102564,1.538462,3.0
13856,28.0,8.0,146.0,6650.0,16100.0,20.0,161.0,41.304348,1.863354,3.0
13857,28.0,8.0,147.0,6650.0,16100.0,20.0,161.0,41.304348,1.863354,3.0
13858,28.0,8.0,148.0,6650.0,15200.0,20.0,152.0,43.750000,1.973684,3.0


#### モデル性能評価用のデータフレーム作成(result_comごとの閾値を持つ)


In [17]:
#学習データの読み込み
result_df = pd.read_csv('../csv/train_{}_date.csv'.format(place))
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除


df=result_df.copy()

#学習、テストデータ切り分け(dateを使って学習データを切り分ける。)

df['date']=pd.to_datetime(df['date'])#文字列なのでdateを日付型に変換
df['year']=df['date'].dt.year
valid_df = df[(df['year']==2019) | ((df['year']==2020) )]#2019,2020のデータを検証用データに。
train_df =  df[(df['year']!=2019) & ((df['year']!=2020) )]#そのほかを学習データに
#学習データを切り分けたらdateとyearはいらないから削除する
valid_df=valid_df.drop(['year'],axis=1)
train_df=train_df.drop(['year'],axis=1)
valid_df=valid_df.drop(['date'],axis=1)
train_df=train_df.drop(['date'],axis=1)

train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])

#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
display(result_com_df)

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1,308,7.277883,1214.480519,920.0
1,2,201,4.749527,1578.159204,1060.0
2,3,162,3.827977,1594.753086,1235.0
3,4,96,2.268431,3353.333333,2010.0
4,5,183,4.324197,1481.092896,1150.0
...,...,...,...,...,...
113,116,3,0.070888,22110.000000,24180.0
114,117,5,0.118147,17042.000000,14130.0
115,118,8,0.189036,24545.000000,23980.0
116,119,5,0.118147,24286.000000,21720.0


### モデル性能評価機能

In [18]:
#基準をクリアしたモデルを格納するdf
depths_arr=[4,5,6,7,8]
model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for target_com in result_com_df['result_com'].values:
    target_com_df=score_df[score_df['target_com']==target_com]
    #閾値の作成===============================================================
    gain_th=110#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==target_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    
    diff_gain_th=108#利益率の閾値
    diff_buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#前後モデルの買ったうちの的中率の閾値
    diff_num_tp_th=result_s['result_com_num'].values[0]*0.13#前後モデルのあたった回数の閾値(出現回数の20%が的中)
    
    #==========================================================
    for target_depth in depths_arr:
        target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]
        
        len_df=len(target_com_depth_df)
        df=target_com_depth_df.copy()
        df['number_i']=np.arange(0,len_df,1)
        
        for _, row in df.iterrows():
            if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
                pass
            else:
                #前後のモデルのスコア===================================
                diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                #===================================================
                if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                    #前後モデルの評価
                    flag_arr=[0,0,0,0]
                    for index in range(len(flag_arr)):

                        diff_model=diff_models[index].iloc[0,:]
                        if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                            flag_arr[index]=1
                        else:
                            pass
                    if sum(flag_arr)==4:
                        model_para_df=model_para_df.append(row)
                        
                else:
                    pass
        

In [19]:
model_para_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
1162,3.0,5.0,124.0,41360.0,36600.0,162.0,366.0,113.005464,13.114754,48.0,73.0
1163,3.0,5.0,125.0,35510.0,31600.0,162.0,316.0,112.373418,13.291139,42.0,74.0
3834,8.0,7.0,123.0,96290.0,79300.0,95.0,793.0,121.424968,4.539723,36.0,72.0
3935,8.0,8.0,125.0,94000.0,80500.0,95.0,805.0,116.770186,4.472050,36.0,74.0
3936,8.0,8.0,126.0,96230.0,77700.0,95.0,777.0,123.848134,4.761905,37.0,75.0
5231,11.0,6.0,134.0,23640.0,20400.0,84.0,204.0,115.882353,10.294118,21.0,83.0
5331,11.0,7.0,135.0,29780.0,26900.0,84.0,269.0,110.706320,9.293680,25.0,84.0
5332,11.0,7.0,136.0,25980.0,23300.0,84.0,233.0,111.502146,9.012876,21.0,85.0
5333,11.0,7.0,137.0,26460.0,23800.0,84.0,238.0,111.176471,9.663866,23.0,86.0
5334,11.0,7.0,138.0,25910.0,21300.0,84.0,213.0,121.643192,10.798122,23.0,87.0


In [20]:
#model_para_df.to_csv('../csv/{}_good_score_model_date.csv'.format(place), encoding='utf_8_sig')
model_para_df.to_csv('../csv/{}_good_score_model_normal.csv'.format(place), encoding='utf_8_sig')

In [21]:
diff_model

target_com       24.00000
depth             7.00000
target_per      137.00000
総収益           14260.00000
投資金額          11300.00000
出現数              39.00000
購買予測数           113.00000
利益率             126.19469
購買的中率             6.19469
的中数               7.00000
number_i         86.00000
Name: 11768, dtype: float64

In [17]:
len_df

99

In [34]:
type(diff_model)

tuple